<div align="right"><i>Peter Norvig<br>2012<br>updated 2019, 2023</i></div>

# Weighing Twelve Balls on a Scale: ①②③④ ⚖ ⑤⑥⑦⑧ 

Here is [a classic](https://en.wikipedia.org/wiki/Balance_puzzle) brain-teaser puzzle from 1945:  

- *You are given twelve identical-looking balls and a two-sided balance scale. One of the balls is of a different weight, although you don't know whether it is lighter or heavier. How can you use just three weighings of the scale to determine not only what the different ball is, but also whether it is lighter or heavier?*


However I want to solve not just this specific puzzle, but related puzzles where you can vary the number of balls; vary the number of weighings allowed; or vary the possibilities for the odd ball: maybe it can only be lighter, not heavier. Or maybe it is  possible that all the balls actually weigh the same. (However, it will always be the case that at most one ball is different.) If I'm going to solve a puzzle with, say, 242 balls, I'd rather use a computer program, not pencil and paper (as was intended for the 1945 version of the puzzle). 

I originally wrote this program in Lisp around 1980, ported it to Python in 2012, and am republishing it here as a notebook after I saw the puzzle was mentioned yet again in the [538 Riddler](https://fivethirtyeight.com/features/which-billiard-ball-is-rigged/) for August 16, 2019. It also appeared in a [Numberplay](https://wordplay.blogs.nytimes.com/2014/07/21/12coin/) column in 2014 (with twelve coins instead of balls), in the Museum of Math's [Mind-Benders for the Quarantined!](https://momath.org/civicrm/?page=CiviCRM&q=civicrm/event/info&reset=1&id=1620) series in 2020, and in many other venues.


# Defining the Vocabulary
    
Here are the main concepts:

- **Ball**: A positive integer from 1 to some *N*. 
- **Oddball**: The one ball that is different from the others, and the way it is different: heavier or lighter. I'll use, e.g., `+3` to mean that ball `3` is heavier than the rest, `-3` to mean that ball `3` is lighter than the rest, and `0` ito mean that all the balls actually weigh the same. 
- **Puzzle**: A description of the number of balls, number of allowable weighings, and the possible oddballs.
- **Answer**: Given a specific puzzle with a specific oddball, a correct **answer** means identifying the specific oddball (in the allowable number of weighings).
- **Solution**: A solution to a puzzle is a **strategy tree** that gives a correct answer for all possible oddballs (in the allowable number of weighings).
- **Strategy tree**: A tree whose root is a **weighing** (of some balls on the left versus some balls on the right) and which specifies the three posssible outcomes of the weighing. Each of these outcomes leads to either another weighing, or a leaf nodes saying which oddball is the answer, under the contingency that the given outcomes occured. Every oddball in the puzzle must appear as a leaf somewhere in the tree. 
- **Outcome**: The **outcome** of a **weighing** will be that the weight of the left side is greater than, equal to, or less than the right side. 
- **Partition**: Each weighing partitions the possible oddballs into three subsets, corresponding to the three possible outcomes. 
- **Rule of 3**: The key insight is that one weighing partitions the set of possible oddballs into 3 subsets,  two weighings into 3 x 3 = 9 subsets, three weighings into 3 x 3 x 3 = 27 subsets, and <i>w</i> weighings into  3<sup><i>w</i></sup> subsets. The 12 ball puzzle has 24 possible oddballs, and thus cannot possibly be solved in two weighings, but *might* be solved in three. In general, *w* weighings can never solve puzzles with more than  3<sup><i>w</i></sup> oddballs.


# Implementation of Data Types and Constants

In [1]:
from dataclasses import dataclass
from typing import Union, Literal, Optional, Dict, Set, Tuple, List, Iterable
import random
random.seed(42) # For reproducability

lt, eq, gt = 'lt', 'eq', 'gt'   # The 3 possible outcomes of a weighing 
unreachable = ()                # A branch of a tree that is impossible to reach

Ball    = int                   # Balls are positive integers
Oddball = int                   # Oddballs are positive (heavier), negative (lighter), or zero (same weight) integers
Outcome = Literal[lt, eq, gt]   # Possible outcomes of a weighing
Leaf    = Oddball               # Leaves are oddballs
# A tree can be an weighing, or a leaf oddball, or None to indicate failure, or `unreachable`
Tree    = Union['Weighing', Leaf, None, Literal[unreachable]] 

@dataclass
class Weighing:
    """An interior node of a strategy tree; represents a weighing and the three possible outcomes.
    Calling `partition` returns a `Weighing` where each gt/eq/lt branch holds a set of oddballs; 
    then `solve` replaces each of those sets with a subtree."""
    L: List[Ball]
    R: List[Ball]
    gt: Union[Set[Oddball], Tree]
    eq: Union[Set[Oddball], Tree]
    lt: Union[Set[Oddball], Tree]

class Puzzle:
    """A ball-weighing puzzle. 
    `N`: the number of balls that can be weighed
    `weighings`: the number of weighings allowed
    `odd`: either a collection of oddball numbers, or a string with some of the characters '+-0',
    where '+' means any ball can be heavy, '-' means any ball can be light, 
    and '0' means that it is possible that all the balls actually weigh the same."""
    def __init__(self, N=12, weighings=3, odd='+-'):
        self.N = N
        self.weighings = weighings
        self.odd = odd
        self.balls = range(1, N + 1)
        self.oddballs = (oddballs(odd, N) if isinstance(odd, str) else odd)
    def __repr__(self): return f'Puzzle(N={self.N}, weighings={self.weighings}, odd={self.odd!r})'

# Implementation of Basic Functions

Here are some straightforward utility functions, before we get to the main `solve` function:

In [2]:
def oddballs(odd: str, N: int) -> List[Oddball]:
    """Parse a string containing "+-0" into a list of oddballs."""
    oddballs = [0] if '0' in odd else []
    if '+' in odd: oddballs.extend(+b for b in range(1, N + 1))
    if '-' in odd: oddballs.extend(-b for b in range(1, N + 1))
    return oddballs

def weigh(L, R, oddball) -> Outcome:
    """Weighing the collection of balls `L` against the balls `R`, given the oddball; return gt, lt, or eq."""
    def heavier(L, R) -> bool: return len(L) > len(R) or oddball in L or -oddball in R
    return (gt if heavier(L, R) else lt if heavier(R, L) else eq)
    
def answer(tree, oddball) -> Oddball:
    """What answer does the strategy tree produce for the given oddball?"""
    if isinstance(tree, Leaf):
        return tree
    else:
        outcome = weigh(tree.L, tree.R, oddball)
        return answer(getattr(tree, outcome), oddball)
    
def is_solution(tree, puzzle) -> bool:
    """Does the strategy tree solve the puzzle correctly for all possible oddballs?"""
    return (tree is not None and depth(tree) <= puzzle.weighings and 
            all(answer(tree, oddball) == oddball 
                for oddball in puzzle.oddballs))

def depth(tree) -> int:
    """Maximum depth (number of weighings) in tree."""
    return (0    if isinstance(tree, Leaf) or not tree
            else 1 + max(depth(tree.gt), depth(tree.eq), depth(tree.lt)))

def first(iterable): return next(iter(iterable))

Let's test out some of these functions:

In [3]:
def test1() -> bool:
    puzzle3 = Puzzle(3, 1, odd='+')
    solution3 = Weighing([1], [3], gt=1, eq=2, lt=3)
    assert is_solution(solution3, puzzle3)
    puzzle8 = Puzzle(8, 3) 
    assert str(puzzle8) == "Puzzle(N=8, weighings=3, odd='+-')"
    assert puzzle8.balls == range(1, 9)
    assert puzzle8.oddballs == [1, 2, 3, 4, 5, 6, 7, 8, -1, -2, -3, -4, -5, -6, -7, -8]
    assert weigh([1, 2], [3, 4], -4) == 'gt'
    return True

test1()

True

# Partitions

Now we've got all the pieces we need to find solutions. The first step is to **partition** a collection of oddballs into three subsets by weighing one set of balls against another. We store the three subsets in a `Weighing` tree node:

In [4]:
def partition(L: List[Ball], R: List[Ball], oddballs) -> Weighing:
    "A node that partitions the possible outcomes of weighing L versus R."
    weighing = Weighing(L, R, set(), set(), set())
    for oddball in oddballs:
        outcome = weigh(L, R, oddball)
        getattr(weighing, outcome).add(oddball)
    return weighing

For example, with 12 balls, if we weigh balls [1, 2, 3] on the left versus [10, 11, 12] on the right, then there are 6 ways  the left side can be greater than the right: either 10, 11 or 12 is lighter or  1, 2, or 3 is heavier. Similarly there are 6 ways of getting a `lt` outcome. The remaining 12 possibilities&mdash;balls 4 through 9 being either heavier or lighter&mdash;show up in the `eq` branch:

In [5]:
puzzle12 = Puzzle(12, 3)

partition([1, 2, 3], [10, 11, 12], puzzle12.oddballs)

Weighing(L=[1, 2, 3], R=[10, 11, 12], gt={1, 2, 3, -12, -11, -10}, eq={4, 5, 6, 7, 8, 9, -9, -8, -7, -6, -5, -4}, lt={10, 11, 12, -2, -3, -1})

If this was the first weighing in our strategy tree, could we go on to solve the puzzle? 

**No!** After using one of our three weighings we can only solve the puzzle if every branch of the partition has no more than 3<sup>2</sup> = 9 oddballs, but here we have 12 oddballs in the `eq` branch. We call this a **bad partition**. A good partition is one where no branch has more than 3<sup><i>w</i></sup> oddballs when there are *w* remaining weighings. Being a good partition does not guarantee that the puzzle is solvable from there; but being a bad partition guarantees that it is not solvable.
 

In [6]:
def good_partition(node, w) -> bool:
    "Is this a partition which might lead to a solution in `w` more weighings?"
    return all(len(oddballs) <= 3 ** w for oddballs in (node.lt, node.eq, node.gt))

In [7]:
good_partition(partition([1, 2, 3], [10, 11, 12], puzzle12.oddballs), 2)

False

With 12 balls, weighing 4 balls on each side is a good partition because each of the branches has 8 possibilities, and 8 is less than 3<sup>2</sup>:

In [8]:
partition4 = partition([1, 2, 3, 4], [9, 10, 11, 12], puzzle12.oddballs)
partition4

Weighing(L=[1, 2, 3, 4], R=[9, 10, 11, 12], gt={1, 2, 3, 4, -12, -11, -10, -9}, eq={5, 6, 7, 8, -8, -7, -6, -5}, lt={9, 10, 11, 12, -2, -4, -3, -1})

In [9]:
good_partition(partition4, 2)

True

# Solving Puzzles

So now we have a viable approach to implementing `solve`: it partitions oddballs down to singletons (or gives up and returns `None` if that can't be done in the allowable number of weighings). For each subset of oddballs in a partition, recursively call `solve` on a new subproblem.
   
Much of the work is done by `good_partitions`. It is a generator that yields good partitions (i.e. ones that might lead to a solution). Most of the time the first or one of the first few partitions will be accepted, but if a partition does not lead to a solution, it will keep generating new attempts up to `tries` times (default 300). It prefers shorter lists of balls on each side of the scale, but is not guaranteed to pick the shortest possible lists. After each try it randomly shuffles the balls, in order to get a different partition on the next try. (I note that for puzzles that specify `odd` as a `"+-0"` string,  there's no sense making multiple tries on the first weighing–the only choice that matters is how many balls, `B`, to put on each side, because the balls are undifferentiated at this point. So I try each value of `B` only once.) If I wanted to solve puzzles with thousands of balls, not just dozens to hundreds, I would come back and make `good_partitions` more efficient.

In [10]:
def solve(puzzle) -> Tree:
    "Find a tree that covers all the oddballs in the given number of weighings."
    N, weighings, oddballs = puzzle.N, puzzle.weighings, puzzle.oddballs
    if len(oddballs) == 1:
        return first(oddballs) # One possibile oddball: an answer
    elif len(oddballs) == 0:
        return unreachable      # This branch of the tree will never be reached
    elif weighings == 0:
        return None            # No solution; failure
    else:
        for node in good_partitions(puzzle):
            node.lt = solve(Puzzle(N, weighings-1, node.lt))
            node.eq = solve(Puzzle(N, weighings-1, node.eq))
            node.gt = solve(Puzzle(N, weighings-1, node.gt))
            if None not in (node.lt, node.eq, node.gt):
                return node
        return None            # No solution; failure

def good_partitions(puzzle, tries=100) -> Iterable[Weighing]:
    "Yield random good partitions."
    oddballs, weighings, balls = puzzle.oddballs, puzzle.weighings, list(puzzle.balls)
    if isinstance(puzzle.odd, str):
        tries = 1 # First weighing, undifferentiated balls: only need to try once.
    for _ in range(tries): 
        for B in range(1, 1 + len(balls) // 2):
            L, R = balls[:B], balls[-B:]
            node = partition(L, R, oddballs)
            if good_partition(node, weighings - 1):
                yield node
        random.shuffle(balls)

Here we see that `good_partitions` does its job:

In [11]:
first(good_partitions(puzzle12))

Weighing(L=[1, 2, 3, 4], R=[9, 10, 11, 12], gt={1, 2, 3, 4, -12, -11, -10, -9}, eq={5, 6, 7, 8, -8, -7, -6, -5}, lt={9, 10, 11, 12, -2, -4, -3, -1})

And so does `solve`:

In [12]:
solve(puzzle12)

Weighing(L=[1, 2, 3, 4], R=[9, 10, 11, 12], gt=Weighing(L=[7, 5, 2, 12], R=[1, 11, 9, 3], gt=Weighing(L=[1, 4, 6, 11], R=[12, 5, 3, 9], gt=-9, eq=2, lt=-11), eq=Weighing(L=[1, 2, 3], R=[10, 11, 12], gt=-10, eq=4, lt=()), lt=Weighing(L=[3], R=[1], gt=3, eq=-12, lt=1)), eq=Weighing(L=[7, 4, 5], R=[3, 6, 11], gt=Weighing(L=[10, 5], R=[7, 3], gt=5, eq=-6, lt=7), eq=Weighing(L=[1, 2, 3, 4, 5], R=[8, 9, 10, 11, 12], gt=-8, eq=(), lt=8), lt=Weighing(L=[1, 11, 12, 2, 5], R=[7, 8, 3, 4, 9], gt=-7, eq=6, lt=-5)), lt=Weighing(L=[8, 6, 3, 9], R=[5, 1, 2, 11], gt=Weighing(L=[5, 4], R=[1, 9], gt=-1, eq=-2, lt=9), eq=Weighing(L=[10, 11, 6], R=[12, 8, 3], gt=10, eq=-4, lt=12), lt=Weighing(L=[1, 2], R=[11, 12], gt=(), eq=-3, lt=11)))

OK, that's really hard to read–my bad. Let's look at a much easier puzzle: 3 balls, 1 weighing allowed, and the oddball can only be heavier:

In [13]:
solve(Puzzle(3, 1, odd='+'))

Weighing(L=[1], R=[3], gt=1, eq=2, lt=3)

This trivial tree says you weigh the first ball against the third (leaving the second unweighed), and the three possible weighing outcomes tell you which of the three balls  is heavier. 

# Prettier Output

Let's make the output easier to read. The function`do(puzzle)` will solve the puzzle, verify that the tree is valid (if it is not `None`), and print the tree in a pretty format using the function `pretty_str(tree)`:
- Balls number 1 to 50 are represented as the Unicode characters `①` to `㊿`, and higher numbers by various other Unicode characters, mostly ball-like.
- A string like `①②③④ ⚖ ⑨⑩⑪⑫ ➔` represents a weighing;
- The three branches of a tree are represented by the prefixes ">:" or "=:" or "<:" (on indented lines, unless they are all leaves).
- The empty tree, representing the fact that all balls balance is represented as "≡".

In [14]:
def do(puzzle):
    "Solve the puzzle; verify the solution; and print the solution in a pretty format."
    tree = solve(puzzle)
    assert (tree is None) or is_solution(tree, puzzle)
    print(pretty_str(tree))
    
def pretty_str(tree, depth=0, prefix='') -> str:
    "Pretty, indented string representing a strategy tree."
    if tree in (None, 0, unreachable):
        return f'{prefix}{tree}'
    elif isinstance(tree, Leaf):
        return f'{prefix}{"+" if tree > 0 else "-"}{ballstr([abs(tree)])}'
    else:
        subtrees = (pretty_str(tree.gt, depth + 1, '>:'), 
                    pretty_str(tree.eq, depth + 1, '=:'), 
                    pretty_str(tree.lt, depth + 1, '<:'))
        indent   = ('' if depth == 0 else ('\n' + '   ' * depth))
        return f'{indent}{prefix}{ballstr(tree.L)} ⚖ {ballstr(tree.R)} ➔ {" ".join(subtrees)}'
    
def ballstr(balls: List[Ball]) -> str:
    """Unicode character string representing a list of balls. The first 50 balls are easy; then it gets a bit wacky."""
    return ''.join(BALLS[i] if (1 <= i < len(BALLS)) else f'{i}' for i in balls)

BALLS = (' ①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴㊵㊶㊷㊸㊹㊺㊻㊼㊽㊾㊿'
         'ⒶⒷⒸⒹⒺⒻⒼⒽⒾⒿⓀⓁⓂⓃⓄⓅⓆⓇⓈⓉⓊⓋⓌⓍⓎⓏ'
         'ⓐⓑⓒⓓⓔⓕⓖⓗⓘⓙⓚⓛⓜⓝⓞⓟⓠⓡⓢⓣⓤⓥⓦⓧⓨⓩ'
         '⦰⦱⦲⦳⦴⦵⦶⦷⦸⦹⦺⦻⦼⦽⦾⦿⧀⧁⧂⧃⨀⨁⨂⨴⨵⨶⨷⨸⓵⓶⓷⓸⓹⓺⓻⓼⓽⓾')

In [15]:
# The easier puzzle from before: 3 balls, 1 weighing, only heavier balls possible
do(Puzzle(3, 1, odd='+'))

① ⚖ ③ ➔ >:+① =:+② <:+③


In [16]:
# 3 balls, 2 weighings, lighter, heavier or equal balls possible
do(Puzzle(3, 2, '+-0'))

① ⚖ ③ ➔ 
   >:② ⚖ ① ➔ >:() =:-③ <:+① 
   =:② ⚖ ① ➔ >:+② =:0 <:-② 
   <:① ⚖ ② ➔ >:() =:+③ <:-①


Note that if both weighings have the outcome of weighing the same, then the string `=:0` above means that the answer is that no ball is different. 

Also, if the first weighing's outcome is that ① is heavier than ③ , and the second weighing's outcome is that ② is heavier than ①, then the string `>:()` above means that this pair of outcomes is impossible; there can be only one heavier ball, so it can't be that ② > ① > ③.

In [17]:
# The original puzzle with 12 balls and 3 weighings
do(puzzle12)

①②③④ ⚖ ⑨⑩⑪⑫ ➔ 
   >:④⑫① ⚖ ③⑧② ➔ 
      >:① ⚖ ⑫ ➔ >:+① =:+④ <:() 
      =:⑦⑩ ⚖ ②⑪ ➔ >:-⑪ =:-⑨ <:-⑩ 
      <:⑤①⑩④⑨ ⚖ ⑫⑦⑪③⑥ ➔ >:-⑫ =:+② <:+③ 
   =:⑩⑥⑤① ⚖ ⑦③④② ➔ 
      >:③⑫① ⚖ ⑦⑤⑨ ➔ >:-⑦ =:+⑥ <:+⑤ 
      =:①②③④⑤ ⚖ ⑧⑨⑩⑪⑫ ➔ >:-⑧ =:() <:+⑧ 
      <:②③⑫ ⚖ ⑤⑦① ➔ >:-⑤ =:-⑥ <:+⑦ 
   <:⑫⑧③⑦ ⚖ ⑨⑩①⑥ ➔ 
      >:⑨⑩③⑥① ⚖ ⑪②④⑧⑤ ➔ >:() =:+⑫ <:-① 
      =:⑧③⑦ ⚖ ②⑩⑪ ➔ >:-② =:-④ <:+⑪ 
      <:③⑩ ⚖ ①④ ➔ >:+⑩ =:+⑨ <:-③


In [18]:
# A different random solution to the same puzzle
do(puzzle12)

①②③④ ⚖ ⑨⑩⑪⑫ ➔ 
   >:⑩①⑤ ⚖ ④③⑨ ➔ 
      >:① ⚖ ⑫ ➔ >:+① =:-⑨ <:() 
      =:④⑨⑦ ⚖ ②③⑫ ➔ >:-⑫ =:-⑪ <:+② 
      <:①⑪③ ⚖ ④⑫⑤ ➔ >:+③ =:-⑩ <:+④ 
   =:⑧⑪⑥ ⚖ ②③⑤ ➔ 
      >:①④⑫⑪③ ⚖ ⑧②⑦⑤⑨ ➔ >:-⑤ =:+⑥ <:+⑧ 
      =:①②③④⑤⑥ ⚖ ⑦⑧⑨⑩⑪⑫ ➔ >:-⑦ =:() <:+⑦ 
      <:⑩⑨⑥ ⚖ ⑧①④ ➔ >:-⑧ =:+⑤ <:-⑥ 
   <:⑥⑩⑧① ⚖ ⑨④⑦③ ➔ 
      >:⑦⑥ ⚖ ③⑩ ➔ >:-③ =:-④ <:+⑩ 
      =:⑫④① ⚖ ⑪⑧⑦ ➔ >:+⑫ =:-② <:+⑪ 
      <:① ⚖ ⑫ ➔ >:() =:+⑨ <:-①


I note that the traditional answer to the 12-ball puzzle weighs 4 balls on each side of the first weighing, three balls on the second weighing, and 2 balls on the third weighing. But my program is not so orderly. It always weighs 4 balls on the first weighing (because that is the only good partition), but from there it can vary; it won't necessarily minimize the number of balls on each side of the scale, nor make one branch of the tree be symmetric with another branch.

# Other Puzzles with 3 Weighings



We can do **12 balls in 3 weighings** even when we add in the possibility that there is no oddball–that all the balls weigh the same:

In [19]:
do(Puzzle(12, 3, odd='+-0'))

①②③④ ⚖ ⑨⑩⑪⑫ ➔ 
   >:①⑦⑧⑨ ⚖ ⑪⑫④③ ➔ 
      >:②⑨④①⑫ ⚖ ⑩⑧③⑦⑥ ➔ >:+① =:-⑪ <:-⑫ 
      =:①② ⚖ ⑪⑫ ➔ >:+② =:-⑩ <:() 
      <:⑩②⑤③ ⚖ ④⑫⑦⑧ ➔ >:+③ =:-⑨ <:+④ 
   =:⑤①⑧ ⚖ ③⑦④ ➔ 
      >:①②③④⑤ ⚖ ⑧⑨⑩⑪⑫ ➔ >:+⑤ =:-⑦ <:+⑧ 
      =:①②③④⑤⑥ ⚖ ⑦⑧⑨⑩⑪⑫ ➔ >:+⑥ =:0 <:-⑥ 
      <:①②③④⑤ ⚖ ⑧⑨⑩⑪⑫ ➔ >:-⑧ =:+⑦ <:-⑤ 
   <:⑪⑥③ ⚖ ④⑫① ➔ 
      >:⑦⑨ ⚖ ①⑪ ➔ >:-① =:-④ <:+⑪ 
      =:⑨⑤⑥② ⚖ ③④⑦⑧ ➔ >:+⑨ =:+⑩ <:-② 
      <:① ⚖ ⑫ ➔ >:() =:-③ <:+⑫


Can we solve the **13-balls in 3 weighings** puzzle, which has 26 possibilities? After all, 26 is less than 3<sup>3</sup> = 27.

In [20]:
puzzle13 = Puzzle(13, 3, odd='+-')
do(puzzle13)

None


**No**, and the reason is that there is no first weighing that partitions the 26 possibilities into three branches, each of which has 9 or fewer possibilities (and thus can be handled in the two remaining weighings). No matter what we do, there will always be a branch with 10 or more possibilities:

In [21]:
partition([1, 2, 3, 4], [10, 11, 12, 13], puzzle13.oddballs)

Weighing(L=[1, 2, 3, 4], R=[10, 11, 12, 13], gt={1, 2, 3, 4, -13, -12, -11, -10}, eq={5, 6, 7, 8, 9, -9, -8, -7, -6, -5}, lt={10, 11, 12, 13, -2, -4, -3, -1})

In [22]:
partition([1, 2, 3, 4, 5], [9, 10, 11, 12, 13], puzzle13.oddballs)

Weighing(L=[1, 2, 3, 4, 5], R=[9, 10, 11, 12, 13], gt={1, 2, 3, 4, 5, -13, -12, -11, -10, -9}, eq={6, 7, 8, -8, -7, -6}, lt={9, 10, 11, 12, 13, -2, -5, -4, -3, -1})

I'll define `partition_sizes` to make this more clear:

In [23]:
def partition_sizes(puzzle, B) -> List[int]:
    "How big is each branch of a partition with B balls on both sides?"
    node = partition(puzzle.balls[:B], puzzle.balls[-B:], puzzle.oddballs)
    return [len(oddballs) for oddballs in (node.gt, node.eq, node.lt)]

In [24]:
{B: partition_sizes(puzzle13, B) for B in range(1, 7)}

{1: [2, 22, 2],
 2: [4, 18, 4],
 3: [6, 14, 6],
 4: [8, 10, 8],
 5: [10, 6, 10],
 6: [12, 2, 12]}

We can do **27 balls in 3 weighings** if we know that the oddball can only be lighter, not heavier (or vice-versa):

In [25]:
do(Puzzle(27, 3, '-'))

①②③④⑤⑥⑦⑧⑨ ⚖ ⑲⑳㉑㉒㉓㉔㉕㉖㉗ ➔ 
   >:⑫⑭⑥④⑩⑱㉒⑦①⑤㉓⑳ ⚖ ③⑮⑲⑰⑬⑨㉔⑧⑯⑪②㉖ ➔ 
      >:⑯㉓②㉑⑭⑮⑪⑦⑰⑬⑲ ⚖ ⑫⑳㉕①④⑥㉗⑱㉖⑤⑨ ➔ >:-㉖ =:-㉔ <:-⑲ 
      =:⑯⑲⑤⑥㉒⑰⑳⑭⑫㉕ ⚖ ㉗①㉔③⑮④⑦⑨⑬② ➔ >:-㉗ =:-㉑ <:-㉕ 
      <:㉖⑫⑤②⑮㉗㉔㉓ ⚖ ⑬㉕⑦③①⑯⑳㉑ ➔ >:-⑳ =:-㉒ <:-㉓ 
   =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗ ➔ 
      >:⑥㉑⑧④②③⑲⑯ ⚖ ㉕㉓⑤⑨㉗⑰⑫⑦ ➔ >:-⑰ =:-⑱ <:-⑯ 
      =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗ ➔ >:-⑮ =:-⑭ <:-⑬ 
      <:㉓㉗⑬⑥⑨⑧⑳⑲⑩ ⚖ ③⑭⑤⑮㉒㉕㉑②⑫ ➔ >:-⑫ =:-⑪ <:-⑩ 
   <:③㉔⑬⑨②⑩⑱⑰㉖ ⚖ ⑧⑳①㉒⑪⑭④㉕⑮ ➔ 
      >:㉓⑥⑦④㉗㉔⑫㉑⑰ ⚖ ①⑨㉕⑪⑭②⑳③⑮ ➔ >:-① =:-⑧ <:-④ 
      =:㉓⑯⑬⑥⑪⑳ ⚖ ⑦㉒⑲㉑㉖⑨ ➔ >:-⑦ =:-⑤ <:-⑥ 
      <:⑬㉓⑪㉕⑰⑮② ⚖ ⑨㉖㉑⑦④㉔⑳ ➔ >:-⑨ =:-③ <:-②


And we can do **26 balls** under the condition that either one ball is lighter or all the balls weigh the same:

In [26]:
do(Puzzle(26, 3, '-0'))

①②③④⑤⑥⑦⑧⑨ ⚖ ⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
   >:㉖⑲⑫⑤⑱ ⚖ ㉔㉓⑨⑳⑥ ➔ 
      >:⑰⑩②⑨⑦㉔⑧⑭⑲㉖⑬ ⚖ ㉓③⑤㉒⑯⑮⑥㉑⑪⑱⑫ ➔ >:-㉓ =:-⑳ <:-㉔ 
      =:⑩⑧⑭㉖㉑ ⚖ ㉓㉕⑱⑰③ ➔ >:-㉕ =:-㉒ <:-㉑ 
      <:⑩㉔⑫㉒⑦⑧⑱ ⚖ ⑮⑲④⑯⑭㉑㉓ ➔ >:-⑲ =:-㉖ <:-⑱ 
   =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
      >:⑦⑱⑫⑭⑨④⑰②⑥ ⚖ ⑮⑤㉓⑪⑳㉔㉑⑲㉒ ➔ >:-⑮ =:-⑯ <:-⑰ 
      =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:-⑭ =:0 <:-⑬ 
      <:①⑳⑨⑬⑱⑥⑯⑮⑪ ⚖ ⑲⑫⑦㉒㉕④㉑⑰③ ➔ >:-⑫ =:-⑩ <:-⑪ 
   <:㉖⑦⑳⑤㉒③⑩⑲ ⚖ ①⑧⑬⑱㉔㉕④㉓ ➔ 
      >:⑤⑳⑨⑧⑱ ⚖ ④⑩⑭㉑㉓ ➔ >:-④ =:-① <:-⑧ 
      =:⑩⑲㉖⑬⑳⑤⑪⑥ ⚖ ⑯⑮㉒㉔⑦④⑨㉕ ➔ >:-⑨ =:-② <:-⑥ 
      <:⑬⑯⑱㉓⑤⑮①⑲⑨⑥㉔ ⚖ ⑦④㉕㉑⑳⑩②㉖㉒⑫⑭ ➔ >:-⑦ =:-③ <:-⑤


Here's a puzzle with **26 balls** where it is stipulated that either one of the odd-numbered balls is heavier, or one of the even-numbered balls is lighter, or there is no oddball.

In [27]:
odd_even = [(+b if b % 2 else -b) for b in range(27)]
print(odd_even)

[0, 1, -2, 3, -4, 5, -6, 7, -8, 9, -10, 11, -12, 13, -14, 15, -16, 17, -18, 19, -20, 21, -22, 23, -24, 25, -26]


In [28]:
do(Puzzle(26, 3, odd=odd_even))

①⑭⑬②③㉔⑳㉑⑥ ⚖ ⑨⑯㉒㉓⑲⑱⑮⑫④ ➔ 
   >:⑪⑥㉓⑦⑮②③⑩⑰ ⚖ ㉑⑯⑨⑤⑬①㉕⑳⑱ ➔ 
      >:⑨⑦①㉔⑯ ⚖ ㉓⑬⑱㉒⑲ ➔ >:-⑱ =:+③ <:-⑯ 
      =:①②③④⑤ ⚖ ㉒㉓㉔㉕㉖ ➔ >:-㉒ =:-⑫ <:-④ 
      <:①②③④⑤⑥ ⚖ ㉑㉒㉓㉔㉕㉖ ➔ >:+① =:+⑬ <:+㉑ 
   =:⑩⑰③㉕⑮④⑫⑱② ⚖ ⑦㉑⑲⑪⑥⑧⑭⑳⑬ ➔ 
      >:③⑰⑮㉔⑯⑬ ⚖ ㉕㉒⑩④⑤⑭ ➔ >:+⑰ =:-⑧ <:+㉕ 
      =:⑮⑭⑫⑰⑥⑨⑳㉒⑩②③ ⚖ ㉖㉔⑬㉕㉓⑪⑱⑦①⑤⑲ ➔ >:-㉖ =:0 <:+⑤ 
      <:①②③④⑤⑥⑦⑧⑨⑩ ⚖ ⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:+⑦ =:+⑪ <:-⑩ 
   <:⑰⑯⑫⑤⑭④⑱⑲⑳ ⚖ ⑪㉔⑩⑬⑮㉕②⑦③ ➔ 
      >:①②③ ⚖ ㉔㉕㉖ ➔ >:-㉔ =:+⑲ <:-② 
      =:㉔⑤⑦⑫⑰ ⚖ ⑨①㉕⑥⑧ ➔ >:-⑥ =:+㉓ <:+⑨ 
      <:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:-⑳ =:-⑭ <:+⑮


Another variation: **27 balls, 3 weighings**, the oddball can only be heavier, but one ball (number 27) is poisonous and can't be touched or placed on the balance scale. It might, however be the heavier ball, and you still need to report it as such if it is. 

We describe this situation by defining a puzzle with 26 (weighable) balls, but with the oddballs including the positive ball numbers from +1 to +27, inclusive. Note that when all three weighings are equal, the strategy tree correctly reports `+㉗`.

In [29]:
do(Puzzle(26, 3, odd=range(1, 28)))

①②③④⑤⑥⑦⑧⑨ ⚖ ⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
   >:⑬⑯⑩④㉕②⑧ ⚖ ①⑱⑤⑮㉔㉓⑥ ➔ 
      >:⑭②⑰ ⚖ ⑧⑳⑥ ➔ >:+② =:+④ <:+⑧ 
      =:⑱②⑦⑳⑪⑰㉕ ⚖ ③⑭⑩④⑯①⑫ ➔ >:+⑦ =:+⑨ <:+③ 
      <:⑧⑳⑤ ⚖ ㉖⑩① ➔ >:+⑤ =:+⑥ <:+① 
   =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
      >:⑧⑮⑳②⑬⑰⑨⑱㉕⑪ ⚖ ①⑭⑥⑯㉒㉔④⑫⑲㉑ ➔ >:+⑪ =:+⑩ <:+⑫ 
      =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:+⑬ =:+㉗ <:+⑭ 
      <:㉖⑪⑬⑤⑰ ⚖ ⑯⑲㉓⑦⑧ ➔ >:+⑰ =:+⑮ <:+⑯ 
   <:⑫⑤⑱⑪㉒㉑ ⚖ ㉔⑳㉖⑯③⑬ ➔ 
      >:㉖③㉑⑨ ⚖ ㉒⑳⑦⑰ ➔ >:+㉑ =:+⑱ <:+㉒ 
      =:④③⑫㉕⑪⑩㉖ ⚖ ㉓⑦⑧㉑⑳②⑤ ➔ >:+㉕ =:+⑲ <:+㉓ 
      <:②⑦⑳⑯㉕⑱① ⚖ ㉖⑭⑪⑰⑬㉒㉑ ➔ >:+⑳ =:+㉔ <:+㉖


# Puzzles with 4 Weighings

We can tackle larger puzzles. With 4 weighings, we can theoretically handle up to 3<sup>4</sup> = 81 possibilities. Can we solve for **40 balls**, heavy or light?

In [30]:
p40 = Puzzle(40, 4)

do(p40)

None


Unfortunately, **no**, we didn't find a solution. That's because the `partition_sizes` always leaves us with a branch of size 28 or more, which can't be handled in the three remaining weighings:

In [31]:
partition_sizes(p40, 13)

[26, 28, 26]

In [32]:
partition_sizes(p40, 14)

[28, 24, 28]

How about **39 balls, 4 weighings**, heavier or lighter, with the possibility that no ball is odd? That's 39 &times; 2 + 1 = 79 possibilities, which is less than 81, and the partition sizes are all 27 or less.

In [33]:
p39 = Puzzle(39, 4, '+-0')

In [34]:
partition_sizes(p39, 13)

[26, 27, 26]

In [35]:
do(p39)

①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴ ➔ 
   >:⑩㉑⑰㉕⑬⑱㊴⑦㉛㉗⑭㉞㊱④ ⚖ ㉒㉘⑲㉚⑥⑮⑨㊲⑪⑯①㉝㉔㉖ ➔ 
      >:㉟⑦㊳⑧㉘㊴⑰㊲⑨⑬ ⚖ ②⑤⑫④⑭⑱㊱⑯①⑥ ➔ 
         >:①②③④⑤⑥⑦ ⚖ ㉝㉞㉟㊱㊲㊳㊴ ➔ >:+⑦ =:+⑬ <:() 
         =:⑥⑩㉘⑯③⑰⑧⑨㉙㉓㉝ ⚖ ㉟㊲④①⑬⑲⑮⑫㊳㉜㉞ ➔ >:+⑩ =:-㉚ <:-㉝ 
         <:⑱㉜①㉛㉗⑬⑮㉖⑥㊳㉚ ⚖ ④㉝⑯⑲㉟㉘⑤㉕㉔㉑⑳ ➔ >:-㉘ =:-㊲ <:+④ 
      =:㉛㉘⑦⑭㉖⑤⑯㉜⑫⑩⑮㉞⑬⑧㉙ ⚖ ②㉚㉑④⑳⑪⑥①⑲㉓⑱㉗㉕㉝㊱ ➔ 
         >:㉟③②㉒㊴㉙㉓⑬㉚⑰㉝⑱⑲⑧ ⚖ ㉑㉛⑮㉕㉔㊱⑳㉗④⑫㊳⑪⑩㊲ ➔ >:+⑧ =:+⑤ <:+⑫ 
         =:㉓⑮㉒④㉘㉟ ⚖ ㉗㉑㊳㉙㉝⑥ ➔ >:-㊳ =:+③ <:-㉟ 
         <:⑩㉓①㉕㊱⑫㉗㊴⑦㉟④㉑⑧⑳㉜ ⚖ ㉔㊳⑯③⑪㉛⑤㉖⑲㉘⑮⑥㉝㊲㉙ ➔ >:-㉙ =:+② <:-㉜ 
      <:⑭⑫㉕⑩㉚㉖⑲㊳⑱㉔㉙㊱㉜②㉗⑮⑥ ⚖ ㉟①㉛㉞㉘⑳㉑③⑧⑯㉓㉝㉒⑤㊲④⑬ ➔ 
         >:⑭㉗㊲㉜㊴㉟⑨⑬㉔㉘⑮③ ⚖ ⑥④㉝㉕㉛②⑧⑫⑤㊳㉖⑱ ➔ >:-㉛ =:-㉞ <:+⑥ 
         =:㉙⑧㉑⑲⑤㉜㉗⑩④㉛⑬⑦ ⚖ ⑪⑮㉞②㉖㊲⑰㊴㉒㊳㉓⑫ ➔ >:-㊴ =:+⑨ <:+⑪ 
         <:㉔㉛ ⚖ ①㉗ ➔ >:-㉗ =:-㊱ <:+① 
   =:⑦㉑③⑩㉛⑰㉞②㊲⑬⑨⑤㉙㉗ ⚖ ⑳㉕⑱⑪㉜㉘⑮④㉔㉒㉚⑭㊱㉝ ➔ 
      >:⑬⑩㉖⑰⑥⑧⑫㉛㉔⑨⑳㉚⑲㉕ ⚖ ㊴④㉗㉜㉒㉘㉝⑭⑤①㊲㉟⑪⑦ ➔ 
         >:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰ ⚖ ㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴ ➔ >:+⑰ =:-㉒ <:-⑭ 
         =:④㉕㊱⑨㉚㉗⑪⑭⑮①⑬㉘㉛㉙⑥㉟ ⚖ ⑱⑫㉖⑲⑤㉜㊲⑳⑩②㉓⑯⑰㊴⑧㉔ ➔ >:-⑱ =:+㉑ <:-⑮ 
         <:㉔㉚⑯⑲ ⚖ ㉕②㉑㉘ ➔ >:-㉕ =:-⑳ <:-㉔ 
      =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰ ⚖ ㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴ ➔ 
         >:㉛㉗⑧㊱⑩

How about **80 balls, 4 weighings** under the condition that no ball can be heavier (thus, 81 possibilities, the maximum)?

In [36]:
do(Puzzle(80, 4, '-0'))

①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗ ⚖ ⒹⒺⒻⒼⒽⒾⒿⓀⓁⓂⓃⓄⓅⓆⓇⓈⓉⓊⓋⓌⓍⓎⓏⓐⓑⓒⓓ ➔ 
   >:ⓑ⑪㉒Ⓟ㊻㉕㉟㊷㊼⑩㉓Ⓥ⑰Ⓛ㉔ⓓ⑦Ⓨ⑥②ⒸⒾ⑱⑮⑤ⒼⓈ ⚖ ㉜ⓍⒿ㊴Ⓓ㊵㊳⑯㊶①㊺ⓇⒷⓏ㉗㊽ⓒⒽⓀ⑧㊱⑲⑳Ⓔ㉞㊾⑫ ➔ 
      >:③㉖㊶㉒Ⓨⓓⓒ⑰㊺⑲Ⓘ㊼⑬Ⓜ②⑫㊾㉝⑤㉗Ⓝ⑥⑪⑯⑧㉑ⒹⒻⒶⒽ㊸㊽ ⚖ Ⓡ㉛㊴⑨④㉕㉜㉓㊻㉚⑱⑦㉘Ⓟⓑ㊵㊲Ⓢ㊱⑩㉞㉔ⓀⓐⒼ①ⒺⓌ⑳⑭㊳㉙ ➔ 
         >:ⓊⒾ㉑Ⓑ㉟㊼⑥㊽Ⓞ㊺⑭⑮㉒Ⓠ㊾ⓐ㊷㊵㉔⑳⑲㉖㊴④Ⓙ㉙ⒻⓉⓓⒺ ⚖ Ⓝ①Ⓩ⑫⑬㊿ⓅⓋⓇ⑩㉝Ⓜ㉗⑪⑯㊶㊹㊲㉞⑱Ⓒ㉜Ⓢ⑰ⓌⓒⒹⒽ㉚ⓑ ➔ >:-Ⓡ =:-Ⓚ <:-Ⓔ 
         =:⑯Ⓖ㊲⑬㉒Ⓚ⑨⑱Ⓣ⑩ⒹⓍ ⚖ ⑭Ⓙ⑳ⓅⓈ㊵⑮Ⓐⓒ㊶㉞④ ➔ >:-Ⓙ =:-Ⓩ <:-Ⓧ 
         <:⑯㊶㊲㊷ⒽⓊ㉗㊽ⒷⓈ⑩㉜㉑⑱㉝㊵ⒶⓂ㉖① ⚖ ⓒⓅ㉙Ⓡ㊾ⓑⓐⓌⓍ⑧⑪㊴㊿㊱㉟⑤Ⓝ③⑮Ⓘ ➔ >:-ⓒ =:-Ⓓ <:-Ⓗ 
      =:㉝㊻㉖⑨⑦㊿⑭ⓁⓈⓌ①Ⓡ㊵ⒷⓅⓐ⑲Ⓙ㊳⑳⑥㉞㊷⑮㉗㉕⑧ⓒⓉ ⚖ ㊱⑪Ⓞ⑩Ⓗ㉓ⓏⓃⓋ⑱㊸Ⓕ㊺Ⓔ㊲㊾㊶㊴④ⒾⓀ㉚㉜⑯⑫ⓓ㊹⑬③ ➔ 
         >:㊱①⑫ⓏⓋⓃ ⚖ ㉔㉕⑪Ⓨ㉙Ⓞ ➔ >:-Ⓞ =:-Ⓕ <:-Ⓝ 
         =:ⓄⓂⓉⒹ㉕㉘㉖㊴㉑Ⓢ㊳㊵ ⚖ Ⓤ⑫⑳①Ⓝ㊺㉙ⓓ㉝ⓍⒿⓇ ➔ >:-Ⓤ =:-Ⓠ <:-Ⓜ 
         <:㉝㉞ⒾⓃⓉⓆⓓ⑮ⒻⓅ⑳㉑Ⓙ⑩Ⓖ㊾⑬㉙⑯Ⓩ㉓㊺Ⓢ⑧Ⓗ⑫㉚④㉘ⒷⓁⓀ㊲Ⓤ② ⚖ Ⓦ㉒㊵㊼㊳⑨㉔㉖⑰⑤㊱⑭㊴ⓄⒸ㉟㊷⑥⑦Ⓐ㊶㊿Ⓡ㊸㉛ⓑⓎ㉕Ⓓ①Ⓥ㊹ⓍⒺ㉗ ➔ >:-Ⓦ =:-ⓐ <:-Ⓣ 
      <:㊷㊶Ⓐ⑳Ⓟ㊵㉞㊹㉚Ⓙ㉒㉙ⓓ㉑⑬Ⓛ㊲ ⚖ Ⓘ㊳ⓊⓍ㊺⑧⑮⑱ⓀⓋⓑ③㉗①㊼㉛⑫ ➔ 
         >:Ⓣ㉒⑰㉞㉟㊷㉙ⓏⒹ㊼ⓊⒻ⑱⑨⑯⑩㉗ⓒ㊾Ⓨⓐ㉛⑧㉔④⑥㉖ⒾⓓⒸ ⚖ ⓋⒿⓈ⑲ⓍⓌ②㊱⑫⑪㊹ⒷⓆⓇ⑭㊴⑬㉑Ⓚ㊿Ⓜ㉕Ⓝ㊺㊽ⒼⓅ㊻⑤㉜ ➔ >:-Ⓥ =:-ⓑ <:-Ⓘ 
         =:ⓓⓁ⑨ⓍⓏⒹ㊽⑯Ⓑ②ⓆⒶ㉜Ⓨ⑮㉑㉚Ⓞ㊱Ⓥ㊲⑲㊹㉔ⓇⓉ㊿ⒾⒿ①㊴㉕㊵㊺㊷Ⓟ ⚖ ⓈⒽ④⑫⑰㊸⑭㉞㉗㉙㊻ⓒ⑳Ⓜ㉖⑱ⓑ⑩⑥㉓㉝㉟Ⓔ③Ⓤ⑦㉘ⓀⒻ㊼⑧㊾ⓐ⑤Ⓦ㊶ ➔ >:-Ⓢ =:-Ⓖ <:-Ⓨ 
         <:ⓄⓈ㊾㊷Ⓦ⑭Ⓥ㉔Ⓗ㉞Ⓔ⑰㊺ⓓ ⚖ Ⓖ㉓Ⓙ㉖㉑⑦㉝Ⓚ㊿㊼ⒸⒶⓁ⑲ ➔ >

Looking good (except that the output is still rather hard to read).

# Puzzles with 5 Weighings

With 5 weighings, 3<sup>5</sup> = 243, so I might be able to handle up to 121 lighter-or-heavier balls (242 possibilities):

In [37]:
puzzle5 = Puzzle(121, 5)
do(puzzle5)

None


Nope, that didn't work. Let's check the partition sizes (they need to be 81 or less to solve in 4 more weighings):

In [38]:
partition_sizes(puzzle5, 40)

[80, 82, 80]

In [39]:
partition_sizes(puzzle5, 41)

[82, 78, 82]

Let's back off to **120 balls, 5 weighings** for 2 &times; 120 + 1 = 241 possibilities:

In [40]:
do(Puzzle(120, 5, '+-0'))

①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴㊵ ⚖ ⓔⓕⓖⓗⓘⓙⓚⓛⓜⓝⓞⓟⓠⓡⓢⓣⓤⓥⓦⓧⓨⓩ⦰⦱⦲⦳⦴⦵⦶⦷⦸⦹⦺⦻⦼⦽⦾⦿⧀⧁ ➔ 
   >:㊴㊳ⓨ⦸⑫⑩④ⓌⒾⓅⓐⓝⓠ㊽⦲①㉛③ⒺⓣⒶⓦ⦹Ⓞⓙ⦶ⓗ⑨㊹Ⓡ㊱⑰Ⓩ㊿Ⓕ⦴②㉞ ⚖ ㉟㉗Ⓑ⑥⑤⑦⑧ⓜ㉓㉑⦻ⓞⒿⓩⓘ⑬⦳㉖ⓓⓤ㊺ⓚⓥ㊼ⓔ⦰㉕⑯ⓢ⦿㊻Ⓖ⑲⦽Ⓛ⑳ⓊⓍ ➔ 
      >:⑤ⓙⓦ㊷⑨㉕㉘⦾ⓋⓩⓕⒶ⦲⑥⦱ⓓ⑬㉞Ⓣ㊵ⓑ⑱㉙⧀㉔ⓞ㊻⑪⑧⦸⦺ⓖⓄⓈ㉓⑩㊸⦷ⓘ㉚㉛ⓝ④⦹ ⚖ ⑫㊳ⓂⓥⓍⓌ㉗㊹⦵②ⓠ㉒⑲⑦ⓨⒽⓛ㊾㊿㉜⦳ⓟⓊ⦿①㊽Ⓓⓒ㉑㊴ⓡ㊶ⓐ㊱⦼ⒾⓏⒿⓤⒺⓁ㉖⦴ⓣ ➔ 
         >:⦹㉚ⓥⓇ①⦱⑧ⓕ㊼Ⓘⓖⓣⓗⓝ㊲㉘㊳⑮Ⓛ㊹Ⓨ㉛ⓜ⑭㉓⧁⑯③ⓔ㊸⑤⦼⦵ⒷⓊⓄ⑨㉜ⓤ ⚖ ㉙Ⓔⓙ⑫㊷ⓦ㉟⦻⑲Ⓒ⧀Ⓟ㊾㉖Ⓩ㊵⦰㉒⑬㊶⑪ⓩⓈ㊿ⓡⓧ⑱㊽㊴⦷④ⓞ⦿⑳Ⓗ⦸㉔ⓘⓛ ➔ 
            >:⑰Ⓞ㉜㊳㉗㊻ⓊⓚⓋ⑥㉘ⓞⓨⓍ㊱ⓦ⑦⦵⦻⧀㉚⑯㊿⑲Ⓒ②㊾①ⓌⓠⓅⓕ㉛ ⚖ ⓤⓎ⧁⑳ⓒⒺ㊸ⓥⒹ㊵⑨⑫㉔㉓⑭⦾㉝ⓀⓈⓟ㉟㊴Ⓖ⦽ⓙⓉ⦶ⓢⓛⓗⓓ⦼㊺ ➔ >:+㉛ =:-⦿ <:+⑨ 
            =:㊹ⓙⓎⓘ⑳ⓋⓝⓃ⑭ⓥ㊱ⓐⓛⓨ㊾⦴⑪㉞㉑Ⓣ⦽ ⚖ ⑩㉙Ⓚ⑧㉕⑫ⓁⓍⒷⒾⓂ⑦⦼㊷ⓞ⦿ⓧ㊸㊵㊺㉓ ➔ >:+㉞ =:-⦳ <:+⑩ 
            <:Ⓞⓣ㉔ⓥⓘⓦ㊼ⓠ⑳⑪Ⓐ⦹㊽Ⓗ⦺ ⚖ ⓤ⦲②⑭⑩⦻⦷Ⓙ㉒Ⓚ㊾⦱㉗㊶ⓖ ➔ >:-ⓤ =:+④ <:-ⓥ 
         =:Ⓢ㊺ⓛⓠⓄⒻ㉗⦾⧁㉓㉞⦷ⓨ㉖ⓕ⑱③Ⓡ㉚㉑Ⓘ⦵ⓑⓒ⦳㊲ⓓ⑲ⓥⓔⓍⓃ⦼㊼ⓧ㊾ⓂⓀⓖ⑦Ⓗⓢ ⚖ ⓐ⦻㊿⑧Ⓥ㊽⑫⦿⦸Ⓙ㊻㊳㉔⑨ⓩ⑭②Ⓠ㉕Ⓛ㊹Ⓩ㉒④ⓦ⦹ⓣ⦴㊶Ⓟ㉙⦲⑬①Ⓑⓘ⦽ⓟⒹⓉ⦶⑰ ➔ 
            >:ⒺⓗⓁⓅ⦹Ⓡ⦾Ⓒ㊱Ⓥ㉚Ⓘ㊽⑲㊺ⒻⓌⓒⓟⓍⓥ㊵Ⓢⓙ⦶ⓠ㉛⧁㉕④㊳㉜Ⓖ㉗⑬⑨ⓛ㊻⦽ ⚖ ⓑ⦻ⓦ㊿⦰㊲ⓤ⦵ⓝ㊸⧀⑪Ⓚ㊾ⓜ⑤Ⓜ⦼⑭ⒹⓊⒶ㉖⦱⑧ⓧⓡ②㊶ⓩⓚⓖ㉔ⓨ⦿㊷Ⓠ⦳㉓ ➔ >:-⦻ =:+③ <:-⦽ 
            =:㉒Ⓨ⦰㊾⑫㉗ⓟ㉚㉔Ⓠⓙⓔ⑮Ⓤ ⚖ ⓜ⑰⦳Ⓒ⑤Ⓞ⦴㊸ⓢ⦶⦱ⓛⓏ② ➔ >:-ⓜ =:-ⓚ <:-⦰ 
            <:㊵Ⓢ㉑㉜⦱ⓥⓄⒽ㊾ ⚖ ⑰ⓢⓨⒹ㉔⦼⦿ⓜ⑨ ➔ >:-ⓢ =:-ⓔ <:+⑰ 
         <:⦼ⓔ㉕ⒺⓠⒶⓈⒿ②ⓕ⑩ⓣ㉘⧀⑭⦷⦺①㊸ⓐ⑨⦸⦲Ⓨⓞ ⚖ ⦰Ⓞ⑮㉜⦽ⓤ㊿

That works.

Or, I could solve a puzzle with 243 possibilites (the maximum possible with 5 weighings): **242 balls**, lighter-only, plus the possibility that no ball is odd. 

This would have printed output with lines twice as long as the previous example, so I won't print the tree, just verify `solve` finds a solution:

In [41]:
puzzle242 = Puzzle(242, 5, '-0')

is_solution(solve(puzzle242), puzzle242)

True

# Puzzles with 6 Weighings

It would be too long and ugly to show the trees for 6 weighings, but I can figure out which puzzles are solvable, and which I fail to solve. 3<sup>6</sup> = 729, and half that is 364.5, so can I solve a 364-ball puzzle?

In [42]:
puzzle364 = Puzzle(N=364, weighings=6, odd='+-')

%time is_solution(solve(puzzle364), puzzle364)

CPU times: user 246 ms, sys: 1.53 ms, total: 247 ms
Wall time: 246 ms


False

**No**, and the fact that it was so fast suggests that there is no good first partition. We can check the partition sizes, which must be no more than 2<sup>5</sup> = 243:

In [43]:
{B: partition_sizes(puzzle364, B) for B in range(118, 124)}

{118: [236, 256, 236],
 119: [238, 252, 238],
 120: [240, 248, 240],
 121: [242, 244, 242],
 122: [244, 240, 244],
 123: [246, 236, 246]}

The best partitions are with 121 or 122 balls on each side, but they leave a 244-oddball branch; one too many.

How about if we drop down to 363 balls, but allow anything for the oddballs (so 2 x 363 + 1 = 727 possible outcomes):

In [44]:
puzzle363 = Puzzle(N=363, weighings=6, odd='+-0')

%time is_solution(solve(puzzle363), puzzle363)

CPU times: user 5.92 s, sys: 3.51 ms, total: 5.92 s
Wall time: 5.92 s


True

**Yes**, there is a solution, and it took 6 seconds to find it. 

One last puzzle: 728 possibly heavy balls and 729 possible outcomes (the maximum for 6 weighings):

In [45]:
puzzle728 = Puzzle(N=728, weighings=6, odd='+0')

%time is_solution(solve(puzzle728), puzzle728)

CPU times: user 21 s, sys: 13.2 ms, total: 21 s
Wall time: 21 s


True

**Yes**, there is a solution, but it took half a minute to solve; I think that's enough.

# What's Next?

- What other puzzles can you solve?
- Can you make a table summarizing the space of solved and unsolved puzzles?
- Can you *prove* the unsolved puzzles are unsolvable? 
- Can you find trees that minimize the *mean* number of weighings, rather than minimizing the *maximum* number of weighings?
- Can you minimize the number of balls in each weighing? The solutions above sometimes weigh 3 or 4 balls on each side of the scale when only 1 or 2 were necessary.
- Currently, when `solve` returns `None`, it means "no solution was found, but there's no proof that a solution doesn't exist." Can you modify `solve` to return a result that indicates there is no possible solution? (Maybe you can only do this in a reasonable amount of time on smaller puzzles, not all.)
- What about puzzles where your task is to identify *which* ball is odd, but not *how* it is odd? That is, if you get the possible oddballs down to `{-3, +3}` then you're done; you know ball 3 is the odd one, and you don't care if it is heavy or light.
- More generally, can you solve puzzles when it is a possibility (or a requirement) that *two* balls are odd?  Or more than two?
- What if you had two or more balance scales that you could use in parallel, and the goal was to minimize the number of weighing time periods, not the total number of weighings?
- What else can you discover?